In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from textblob import TextBlob
from textblob import Word
import spacy
import re,string,unicodedata
from wordcloud import WordCloud,STOPWORDS
import os
print(os.listdir("input"))
import warnings
warnings.filterwarnings('ignore')

['newsgroups.csv']


In [2]:
# load the newsgroups dataset from the CSV file
newsgroups = pd.read_csv('input/newsgroups.csv')

# extract the text content and integer labels
text = newsgroups['data']
labels = newsgroups['target']

# replace NaN values with empty string
newsgroups['data'] = newsgroups['data'].fillna('')

# define a stemmer
stemmer = PorterStemmer()

# set stopwords to english
stop_words = set(stopwords.words('english'))

#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')


In [3]:
target_names = ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 
                'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 
                'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 
                'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 
                'talk.religion.misc']

In [4]:
newsgroups.describe()

,target
count,18846.000000
mean,9.293166
std,5.562798
min,0.000000
25%,5.000000
50%,9.000000
75%,14.000000
max,19.000000


In [5]:
labels.value_counts()

10    999
15    997
8     996
9     994
11    991
7     990
13    990
5     988
14    987
2     985
12    984
3     982
6     975
1     973
4     963
17    940
16    910
0     799
18    775
19    628
Name: target, dtype: int64

In [6]:
def preprocess_text(text):
    if isinstance(text, float) and np.isnan(text):
        return ""
text = text.apply(preprocess_text)

In [7]:
def strip_html(text):
    if text is not None:
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()
    else:
        return ""

#Removing the square brackets
def remove_between_square_brackets(text):
    if text is not None:
        return re.sub('\[[^]]*\]', '', text)
    else:
        return ""

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

# apply the preprocessing to the text data
text = text.apply(denoise_text)

In [8]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
text = text.apply(remove_special_characters)

In [9]:
#Converting text to lowercase
def preprocess_text(text):
    #convert to lowercase
    text = text.lower()
    return text
    
# apply the preprocessing to the text data
text = text.apply(preprocess_text)

In [10]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
text = text.apply(simple_stemmer)

In [11]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
text = text.apply(remove_stopwords)

{"she's", 'by', "haven't", 'who', 'them', "aren't", 'to', 'wouldn', 'isn', "shan't", 'below', "hadn't", 'just', "you've", 'my', 'or', "isn't", 't', 'more', 'having', 'doing', 'whom', 'didn', 'out', 'which', 'here', 'nor', 'she', 's', 'he', 'yours', 'those', 'itself', 'her', 'once', "mustn't", "wouldn't", 'for', 'had', 'that', 'during', 'very', 'both', 'you', 'about', 'we', "didn't", 'now', 'ma', 'some', 'with', 'when', 'doesn', 'ours', 'shan', 'from', 'before', 'our', 'been', 'it', 'aren', 'don', 'mightn', 'couldn', 'your', 'ourselves', 'as', 're', 'against', 'because', "shouldn't", 'further', 'over', 'did', 'and', 'between', 'do', 'all', "should've", 'me', 'most', 'have', 'hers', 'into', 'only', 'were', 'above', 'needn', 'haven', "mightn't", 'being', 'myself', 'than', 'o', 'does', 'how', "couldn't", "you'll", 'own', 'while', 'down', 'hadn', 'under', 'these', 'through', 'm', 'each', 'of', 'is', 'am', "hasn't", 'after', 'their', 'theirs', 'other', 'y', 'be', 'i', 'no', "wasn't", 'this',

In [12]:
text

0         
1         
2         
3         
4         
        ..
18841     
18842     
18843     
18844     
18845     
Name: data, Length: 18846, dtype: object

In [13]:
labels

0        10
1         3
2        17
3         3
4         4
         ..
18841    13
18842    12
18843     3
18844     1
18845     7
Name: target, Length: 18846, dtype: int64

In [14]:
#split the dataset  
#train dataset
train_text=newsgroups.data[:15076]
train_labels=newsgroups.target[:15076]
#test dataset
test_text=newsgroups.data[15076:]
test_labels=newsgroups.target[15076:]
print(train_text.shape,train_labels.shape)
print(test_text.shape,test_labels.shape)

(15076,) (15076,)
(3770,) (3770,)


In [15]:
# split the data into training and testing sets
train_text, test_text, train_labels, test_labels = train_test_split(text, labels, test_size=0.2, random_state=42)
print(train_text.shape,train_labels.shape)
print(test_text.shape,test_labels.shape)

(15076,) (15076,)
(3770,) (3770,)


In [16]:
#normalized train data
norm_train_text=newsgroups.data[:15076]
norm_train_text[0]

"\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [17]:
#Normalized test data
norm_test_text=newsgroups.data[15076:]
norm_test_text[18845]

'After a tip from Gary Crum (crum@fcom.cc.utah.edu) I got on the Phone\nwith "Pontiac Systems" or "Pontaic Customer Service" or whatever, and\ninquired about a rumoured Production Hold on the Formula Firebird and\nTrans Am.  BTW, Talking with the dealer I bought the car from got me\nnowhere.  After being routed to a "Firebird Specialist", I was able\nto confirm that this is in fact the case.\n\nAt first, there was some problem with the 3:23 performance axle ratio.\nShe wouldn\'t go into any details, so I don\'t know if there were some\nshipped that had problems, or if production was held up because they\nsimply didn\'t have the proper parts from the supplier.  As I say, she\nwas pretty vague on that, so if anyone else knows anything about this,\nfeel free to respond.  Supposedly, this problem is now solved.\n\nSecond, there is a definate shortage of parts that is somehow related\nto the six-speed Manual transmission.  So as of this posting, there is\na production hold on these cars.  S

In [18]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed training data
cv_train_text=cv.fit_transform(norm_train_text)
#transformed test data
cv_test_text=cv.transform(norm_test_text)
print('BOW_cv_train:',cv_train_text.shape)
print('BOW_cv_test:',cv_test_text.shape)
#vocab=cv.get_feature_names()-toget feature names

BOW_cv_train: (15076, 2512741)
BOW_cv_test: (3770, 2512741)


In [19]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed training data
tv_train_text=tv.fit_transform(norm_train_text)
#transformed test data
tv_test_text=tv.transform(norm_test_text)
print('Tfidf_train:',tv_train_text.shape)
print('Tfidf_test:',tv_test_text.shape)

Tfidf_train: (15076, 2512741)
Tfidf_test: (3770, 2512741)


In [20]:
#labeling the target data
lb=LabelBinarizer()
#transformed labels data
label_data=lb.fit_transform(newsgroups['target'])
print(label_data.shape)

(18846, 20)


In [21]:
#Splitting the target data
train_labels=label_data[:15076]
test_labels=label_data[15076:]
print(train_labels)
print(test_labels)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
[[0 0 1 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [22]:
train_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [86]:
#converting 2d array to 1d 
#skipping generates error(1d array is expected in LR)
#doing this gets the kernel stuck

train_labels = np.argmax(train_labels, axis=1)

In [87]:
train_labels[0]

10

In [74]:
#training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=0.01)

In [ ]:
#Fitting the model for Bag of words
lr_bow=lr.fit(cv_train_text,train_labels)
print(lr_bow)

In [ ]:
#Fitting the model for tfidf features
lr_tfidf=lr.fit(tv_train_text,train_labels)
print(lr_tfidf)

In [ ]:
#Predicting the model for bag of words
lr_bow_predict=lr.predict(cv_test_text)
print(lr_bow_predict)

In [ ]:
##Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(tv_test_text)
print(lr_tfidf_predict)

In [ ]:
###Experimental below/Don't run

In [ ]:
# create a logistic regression model
lr_clf = LogisticRegression()

# train the model on the training data
lr_clf.fit(X_train,y_train)

# make predictions on the training data
lr_pred = lr_clf.predict(X_train)
train_score = accuracy_score(y_train, lr_pred) * 100
print(f"Train accuracy score: {train_score:.2f}%")

# make predictions on the testing data
lr_pred = lr_clf.predict(cv_test_text)
test_score = accuracy_score(test_labels, lr_pred) * 100
print(f"Test accuracy score: {test_score:.2f}%")

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_clf = GaussianNB()
nb_clf.fit(X_train.toarray(), y_train)

nb_pred = nb_clf.predict(X_train.toarray())
train_score = accuracy_score(y_train, nb_pred) * 100
print(f"Train accuracy score: {train_score:.2f}%")

nb_pred = nb_clf.predict(X_test.toarray())
test_score = accuracy_score(test_labels, nb_pred) * 100
print(f"Test accuracy score: {test_score:.2f}%")

In [ ]:
svm_clf = SVC()
svm_clf.fit(X_train.toarray(), y_train)
print("Training completed")

svm_pred = svm_clf.predict(X_train.toarray())
train_score = accuracy_score(y_train, svm_pred) * 100
print(f"Train accuracy score: {train_score:.2f}%")

svm_pred = svm_clf.predict(X_test.toarray())
test_score = accuracy_score(y_test, svm_pred) * 100
print(f"Test accuracy score: {test_score:.2f}%")

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "/Users/bibeksharma/Downloads/4")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "/Users/bibeksharma/Downloads/4/")
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

x_train = np.random.random((1000, 100))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=5, batch_size=32)

In [ ]:
import sys
import tensorflow as tf
import tensorflow.keras
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "AVAILABLE" if gpu else "NOT AVAILABLE")

In [ ]:
import tensorflow as tf

# Create some tensors to use
a = tf.random.normal([1000, 1000])
b = tf.random.normal([1000, 1000])

# Print the currently used physical device
print("Currently used device:", tf.test.gpu_device_name() or "CPU")


In [23]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [11]:
# create a logistic regression model
lr_clf = LogisticRegression()

# train the model on the training data
lr_clf.fit(X_train,y_train)

# make predictions on the training data
lr_pred = lr_clf.predict(X_train)
train_score = accuracy_score(y_train, lr_pred) * 100
print(f"Train accuracy score: {train_score:.2f}%")

# make predictions on the testing data
lr_pred = lr_clf.predict(cv_test_text)
test_score = accuracy_score(test_labels, lr_pred) * 100
print(f"Test accuracy score: {test_score:.2f}%")

Train accuracy score: 96.91%
Test accuracy score: 77.20%


In [12]:
from sklearn.naive_bayes import GaussianNB
nb_clf = GaussianNB()
nb_clf.fit(X_train.toarray(), y_train)

nb_pred = nb_clf.predict(X_train.toarray())
train_score = accuracy_score(y_train, nb_pred) * 100
print(f"Train accuracy score: {train_score:.2f}%")

nb_pred = nb_clf.predict(X_test.toarray())
test_score = accuracy_score(test_labels, nb_pred) * 100
print(f"Test accuracy score: {test_score:.2f}%")

Train accuracy score: 92.15%
Test accuracy score: 74.20%


In [1]:
print("\n Evaluating Model ... \n")
predicted = model.predict_classes(X_test_Glove)
print(metrics.classification_report(y_test, predicted))
print("\n")
logger = logging.getLogger("logger")
result = compute_metrics(y_test, predicted)
for key in (result.keys()):
    logger.info("  %s = %s", key, str(result[key]))


 Evaluating Model ... 



NameError: name 'model' is not defined

In [2]:
history = model.fit(X_train_Glove, y_train,
                           validation_data=(X_test_Glove,y_test),
                           epochs=5,
                           batch_size=128,
                           verbose=1)


NameError: name 'model' is not defined